<a href="https://colab.research.google.com/github/Jeremy26/neural_optimization_course/blob/main/Mini_Pruning_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Pruning Mini-Workshop

In [ ]:
# pruning header import
import torch.nn.utils.prune as prune
import torch.nn as nn
import matplotlib.pyplot as plt
# pruning operations are done in-place, so take a copy of the model / module
from copy import deepcopy
import numpy as np

In [ ]:
# create a dummy module to understand pruning techniques
fc_test = nn.Linear(10,10)

module = deepcopy(fc_test)

In [ ]:
# contains `weight` and `bias` parameters
print('Before pruning, named_parameters()')
print(list(module.named_parameters()))

print('Before pruning, named_buffers()')
# prior to pruning contains no buffers
print(list(module.named_buffers()))

## L1 Unstructured Pruning

### Numpy

In [ ]:
weight = module.weight.cpu().detach().numpy()
print(weight)

In [ ]:
sorted_array = #TODO: Sort the Weights
print(sorted_array)

In [ ]:
pruned_array = #TODO: Prune the 30% lowest weights
print(pruned_array.astype(int))

### PyTorch

In [ ]:
# l1_unstructured means that weights are pruned according to their `L1_norm`
# eg: the following line prunes 30% of weights in module according to their L1 norm

#TODO: Call L1 Unstructured Pruning on PyTorch

In [ ]:
# contains `weight_orig` and `bias` parameters
print('After pruning, named_parameters()')
print(list(module.named_parameters()))

# after pruning contains `weight_mask`
print('After pruning, named_buffers()')
print(list(module.named_buffers()))

## L1 Structured

### Numpy

In [ ]:
module = deepcopy(fc_test)

weight = module.weight.cpu().detach().numpy()
print(weight)

In [ ]:
# finding the L1 norm for each row (equivalent to 
# total norm of each neuron) and sort them 
sorted_array = #TODO: Sort the Weights
print(sorted_array)

In [ ]:
pruned_array = #TODO: Prune the Weights in Structured Mode

### PyTorch

In [ ]:
module = deepcopy(fc_test)
# n denotes the order of `L-norm` to use while pruning
# dim indicates which dimension to prune

#TODO: Prune 30% Structured with PyTorch

In [ ]:
# We can observe entire rows of weights are set to zero, meaning all 
# connections to a neuron have been pruned
list(module.named_buffers())

## Plot

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(8, 4))

# l1 unstructured pruning
module = deepcopy(fc_test)
prune.l1_unstructured(module, 'weight', amount=0.3);
plot_fc_weight(module, ax1)
ax1.set_title('L1 Unstructured Pruning')

# l1 Structured pruning
module = deepcopy(fc_test)
prune.ln_structured(module, 'weight', amount=0.3, n=1, dim=0);
plot_fc_weight(module, ax2);
ax2.set_title('L1 Structured Pruning');

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(8, 4))
plot_fc_weight(module.weight, ax1)

# l1 unstructured pruning
module = deepcopy(fc_test)
prune.l1_unstructured(module, 'weight', amount=0.3);
plot_fc_weight(module.weight, ax1);
ax1.set_title('L1 Unstructured Pruning')

# l1 Structured pruning
module = deepcopy(fc_test)
prune.ln_structured(module, 'weight', amount=0.3, n=1, dim=0);
plot_fc_weight(module.weight, ax2);
ax2.set_title('L1 Structured Pruning');